<a href="https://colab.research.google.com/github/Eran707/MSc-Computational-Neuroscience-Repo/blob/master/Multicompartment_Sim_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Neural Multicompartment Simulator**

**Eran Frank Shorer**

**MSc Computational Neuroscience**

**"Investigating the effects of impermeant anions on the electrical and computational properties of neurons"**




## Instructions:

This jupyter notebook consists of cells. Some cells are plain text, while others are code. 

Press Shift-Enter simultaneously, or click the cell and press the play button in the main toolbar above to run the cell. 

The output of the cell should be displayed below the cell. 



## **Import classes and modules** 


In [85]:
import pandas as pd

try:
    from ipywidgets import widgets, Layout, interact, interactive, interactive_output, Dropdown
    from IPython.display import display
    import bqplot as bqp
    from bqplot import pyplot as plt # bqplot is a plotting for jupyter
    import matplotlib.pyplot as mplt
    import common
    import constants
    import compartment
    import electrodiffusion
    import numpy as np
    
    import seaborn as sns 
    import pandas as pd
    import graphing as gr
    #!pip install viola
    
    
except ModuleNotFoundError:
        print("A class you are trying to import is not present in the directory")
    
except Exception:
        print("Something went wrong - modules have not been imported")

else: 
        print("All relevant classes imported")
    





All relevant classes imported


## **Set simulation parameters**


## *A) Compartment parameters*

### A1) Set compartment parameters:

In [57]:
### WIDGET INITIALIZATION ################################################################################
txt_name = widgets.Text(value="Comp1")
lbl_name = widgets.Label(value ="Compartment Name: ")
name_box = widgets.HBox(children = [lbl_name,txt_name])


lbl_rad = widgets.Label(value = "Compartment Radius (um): ")
sldr_rad = widgets.FloatSlider(value=5, min=1, max=11.0, step=0.5, disabled=False, continuous_update=False,
    orientation='horizontal', readout=True, readout_format='.1f')
rad_box = widgets.HBox(children = [lbl_rad,sldr_rad])

lbl_len = widgets.Label(value = "Compartment Length (um): ")
sldr_len = widgets.FloatSlider(value=100, min=50, max=200, step=20, disabled=False, continuous_update=False,
    orientation='horizontal', readout=True, readout_format='.1f')
len_box = widgets.HBox(children = [lbl_len,sldr_len])

lbl_nai = widgets.Label(value = "Na+ concentration (mM): ")
sldr_nai = widgets.FloatSlider(value=14, min=0, max=100, step=10, disabled=False, continuous_update=False,
    orientation='horizontal', readout=True, readout_format='.1f')
hbox_nai = widgets.HBox(children = [lbl_nai,sldr_nai])

lbl_ki = widgets.Label(value = "K+ concentration (mM): ")
sldr_ki = widgets.FloatSlider(value=122, min=0, max=250, step=5, disabled=False, continuous_update=False,
    orientation='horizontal', readout=True, readout_format='.1f')
hbox_ki = widgets.HBox(children = [lbl_ki,sldr_ki])

lbl_cli = widgets.Label(value = "Cl- concentration (mM): ")
sldr_cli = widgets.FloatSlider(value=5, min=0, max=50, step=5, disabled=False, continuous_update=False,
    orientation='horizontal', readout=True, readout_format='.1f')
hbox_cli = widgets.HBox(children = [lbl_cli,sldr_cli])

lbl_xi = widgets.Label(value = "Impermeant anion concentration (mM): ")
sldr_xi = widgets.FloatSlider(value=150, min=0, max=300, step=10, disabled=False, continuous_update=False,
    orientation='horizontal', readout=True, readout_format='.1f')
hbox_xi = widgets.HBox(children = [lbl_xi,sldr_xi])

lbl_zi = widgets.Label(value = "Impermeant anion average charge: ")
sldr_zi = widgets.FloatSlider(value=-0.85, min=-3, max=0, step=0.05, disabled=False, continuous_update=False,
    orientation='horizontal', readout=True, readout_format='.1f')
hbox_zi = widgets.HBox(children = [lbl_zi,sldr_zi])

########################################################################################################


global name, comp_arr, df_sim


#CREATION OF THE PANDAS DATA FRAME THAT WILL KEEP TRACK OF ALL THE VARIABLES IN REAL TIME
df_sim = pd.DataFrame()

comp_arr =[]

output1 = widgets.Output()


btn_new_comp = widgets.Button(description = "Create Compartment",button_style ="success")


def new_comp_clicked(b):
    
    name = txt_name.value
    for j in range (len(comp_arr)):
        if name == comp_arr[j].name:
            raise Exception("Two compartments cannot have the same name")
    new_comp = compartment.Compartment(name,radius=sldr_rad.value*1e-5, length=sldr_len.value*1e-5)
    new_comp.set_ion_properties(na_i=sldr_nai.value*1e-3, k_i = sldr_ki.value*1e-3, cl_i=sldr_cli.value*1e-3,x_i=sldr_xi.value*1e-3, z_i=sldr_zi.value)
    comp_arr.append(new_comp)
    arr_df = new_comp.get_df_array()
    df_sim[name] = arr_df
    df_sim.index = ['Radius', 'Length', 'Volume', 'Na_i', 'K_i', 'Cl_i', 'X_i', 'z_i', 'ATPase pump rate','KCC2 pump rate','Vm', 'Ek', 'ECl']
    with output1:
        print("Compartment: ("+ new_comp.name +") created" )
        print(new_comp.get_df_array())
      
btn_new_comp.on_click(new_comp_clicked)


vbox_grand = widgets.VBox(children = [name_box,rad_box,len_box, hbox_nai,hbox_ki,hbox_cli,hbox_xi,hbox_zi,btn_new_comp], layout=Layout(border ="solid"))

display(output1,vbox_grand)








Output()

### A2) Review & edit compartments

In [58]:
btn_review = widgets.Button(description = "Review compartments", button_style ="success")
output2 = widgets.Output()
display(btn_review,output2)

def btn_review_clicked(b):
    output2.clear_output()
    with output2:
        display(df_sim)

btn_review.on_click(btn_review_clicked)       


Button(button_style='success', description='Review compartments', style=ButtonStyle())

Output()

In [4]:
btn_edit = widgets.Button(description = "Edit compartments", button_style ="success")
output3 = widgets.Output()
display(btn_edit,output3)

btn_save_edit = widgets.Button(description = "Save change", button_style ="success",visible='false')


def btn_edit_clicked(b):
    
    output3.clear_output()
    opts_comp =[]
    for i in range(len(comp_arr)):
        opts_comp.append(comp_arr[i].name)

    global drp_comp,drp_param,txt_new_val
    drp_comp=widgets.Dropdown(options=opts_comp,description='Compartment:',visible='false')
    opts_param=df_sim.index.array[0:10]
    drp_param=widgets.Dropdown(options=opts_param, description='Parameter:', visible='false')

    txt_new_val=widgets.Text(value='', description='New value:', visible='false')
     

    
    drp_param.visible ="true"
    drp_comp.visible ="true"
    txt_new_val.visible="true"
    btn_save_edit.visible ='true'
    
    with output3:
        display(drp_comp,drp_param,txt_new_val,btn_save_edit)
  




btn_edit.on_click(btn_edit_clicked)




def btn_save_edit_clicked(b):
    comp = drp_comp.value
    param = drp_param.value
    change = txt_new_val.value
    
    df_sim.loc[param,comp] = change
    output2.clear_output()
    output3.clear_output()
    with output3:
        display(df_sim)

btn_save_edit.on_click(btn_save_edit_clicked)  



Button(button_style='success', description='Edit compartments', style=ButtonStyle())

Output()

### A3) Multicompartment visualization


In [59]:
btn_visualize = widgets.Button(description = "Visualize compartments",button_style ="success")

output4 = widgets.Output()
display(btn_visualize,output4)






def btn_visualize_clicked(b):
    
    
    output4.clear_output(
    ) 
   
    for i in range(df_sim.shape[1]):
        
        hgt = str(df_sim.loc['Length',comp_arr[i].name] *80000)+"px"
        wdth = str(df_sim.loc['Radius',comp_arr[i].name]*5000000)+"px"
        btn = widgets.Button(layout=Layout(height = hgt, width =wdth,border="solid"), disabled =True,description =comp_arr[i].name)
        btn.style.button_color = 'brown'
        with output4:
            display(btn)
        

            

btn_visualize.on_click(btn_visualize_clicked)






Button(button_style='success', description='Visualize compartments', style=ButtonStyle())

Output()

### A4) Electrodiffusion setup

In [60]:
## Loop to run through the compartment array and join them by electrodiffusion
btn_ed = widgets.Button(description="Link compartments with electrodiffusion",button_style ="success", layout=Layout(width="300px"))
output5 = widgets.Output()
display(btn_ed,output5)

ed_arr = [] # array of all the electro-diffusion links between the compartments
ed_dict_arr = [] # array of all the electro-diffusion dictionaries (constantly changing)
ed_conc_changes_arr = []



def btn_ed_clicked(b): #try making this run the compartment class method multi_comp_ed_link()
    for e in range(len(comp_arr)-1):
        ed_arr.append(electrodiffusion.Electrodiffusion(comp_arr[e],comp_arr[e+1]))
    with output5:
        print("All compartments successfully linked")
    btn_ed.disabled=True
    
btn_ed.on_click(btn_ed_clicked)



Button(button_style='success', description='Link compartments with electrodiffusion', layout=Layout(width='300…

Output()

## *B) Timing*

In [61]:
global total_t,dt,t_arr

### TOTAL TIME
lbl_total_t = widgets.Label(value="Set the simulation run time (minutes):")

sldr_total_t = widgets.FloatSlider(value=1, min=0.5, max=10, step=0.5, disabled=False, continuous_update=False, orientation='horizontal',
    readout=True, readout_format='.1f')
HBox_total_t = widgets.HBox(children = [lbl_total_t,sldr_total_t])


### TIME STEP
lbl_dt = widgets.Label(value="Set the simulation time step (milliseconds):")
sldr_dt = widgets.FloatSlider(value=1, min=1, max=5, step=0.2, disabled=False, continuous_update=False, orientation='horizontal',
    readout=True, readout_format='.1f')
HBox_dt = widgets.HBox(children = [lbl_dt,sldr_dt])








    
t_arr = [0]


vbox_t = widgets.VBox([HBox_total_t,HBox_dt], layout=Layout(border='solid'))





display(vbox_t)


#print("Set total simulation time (minutes):")

#print("Set time step (milliseconds):")



## *C) Run Simulation*

In [62]:

btn_sim = widgets.Button(description ="Run Simulation", button_style="success",disabled=False)
output6 = widgets.Output()
output7 = widgets.Output()
display(output6, output7, btn_sim)

global SIM_OVER
SIM_OVER = False

def btn_sim_clicked(b):
  
    global dt,total_t
    dt=sldr_dt.value*1e-3
    total_t= sldr_total_t.value *60
    interval = total_t/dt
                   
    run_t=0
  
    prg = widgets.FloatProgress(description='Simulating...', min=0,step=dt,max=total_t,value=0,continuous_update=True)
    with output6:
        display(prg)
    
    ED_ON = True
    

    while run_t < total_t:

        
        if ED_ON:

            ed_dict_arr = []  # array of all the electro-diffusion dictionaries (constantly changing)
            ed_conc_changes_arr = []


            for a in range(len(comp_arr)):
                comp_arr[a].step(dt)  # step for each compartment
                ed_dict_arr.append(comp_arr[a].get_ed_dict())  # electrodiffusion dictionary for each compartment

            for b in range(len(comp_arr) - 1):
                ed_conc_changes_arr.append(
                    ed_arr[b].calc_ed(dt, ed_dict_arr[b], ed_dict_arr[b + 1]))  # makes an array of all the ED conc changes

            for c in range(len(comp_arr) - 1):
                comp_arr[c].ed_update(ed_conc_changes_arr[c],
                                      "positive")  # appending the electrodiffusion concentrations for each compartment
                comp_arr[c + 1].ed_update(ed_conc_changes_arr[c], "negative")

            for d in range(len(comp_arr)):
                comp_arr[d].update_volumes()  # updates of the volumes, arrays, and dataframe for each compartment
                if run_t != 0:
                    comp_arr[d].update_arrays()
                df_sim[comp_arr[d].name] = comp_arr[d].get_df_array()
            
            if run_t != 0:
                t_arr.append(run_t)
                
            run_t += dt
            prg.value += dt  

        else: # if you want to run with normal diffusion not ED
            for a in range(len(comp_arr)):
                comp_arr[a].step(dt)
                comp_arr[a].update_volumes()  # updates of the volumes, arrays, and dataframe for each compartment
                comp_arr[a].update_arrays()
                df_sim[comp_arr[a].name] = comp_arr[d].get_df_array()
    
    print(total_t)    
    SIM_OVER = True
    btn_sim.disabled=True
    lbl_finalvals = widgets.Label(value="Final Values:")
    with output7:  
        print("Simulation complete!")
        display(lbl_finalvals,df_sim)
        
        




        
btn_sim.on_click(btn_sim_clicked)  



Output()

Output()

Button(button_style='success', description='Run Simulation', style=ButtonStyle())

60.0


## 4) Graphing


In [63]:

SIM_OVER == True ## * Just for testing purposes

#graphing module can onlt be accessed once the simulation is complete

    ##### User interface:
    
drp_comp = widgets.Dropdown(description='Compartment:')

options_drp_comp = []
options_drp_comp.append('ALL COMPARTMENTS')
for i in range(len(comp_arr)):
    options_drp_comp.append(comp_arr[i].name)

drp_comp.options = options_drp_comp    



drp_param = widgets.Dropdown(description='Parameter:')
options_drp_param = ['Membrane Potential (Vm)','Cl- Reversal Potential (E-Cl)', 'K+ Reversal Potential (E-K)',''
                        ,'Volume (pL)','',
                         'All ion concentrations','Na+ Conc.','K+ Conc.','Cl- Conc','Impermeant anion Conc']


drp_param.options = options_drp_param

intervals = len(t_arr) #amount of time points simulated

sldr_start = widgets.FloatSlider(description='Start time (s)',min=0,step=dt,max=total_t, value=20)
sldr_bins = widgets.FloatSlider(description='Bins',min=50,step=50,max=intervals, value=1000)




hbox_graph = widgets.HBox()
vbox_graph1 = widgets.VBox()
vbox_graph2 = widgets.VBox()
vbox_graph1.children =[drp_comp,drp_param]
vbox_graph2.children =[sldr_start,sldr_bins]
hbox_graph.children = [vbox_graph1,vbox_graph2]

btn_main_graph = widgets.Button(description='Graph it!',button_style='success')
output10 = widgets.Output()
display(hbox_graph,btn_main_graph,output10)

def plot_main_graph(param='',comp_num=0,all_comps =False,jump=1):
    
        
        fig = plt.figure()
        fig.axes = []
        fig.legeng_location = 'bottom'
        plt.clear()
        xlabel ="Time (s)"
        ylabel = param
        y_ax_arr = []
        
        start_t = int(sldr_start.value * 1000)
        x_arr = t_arr 
    
        bins = sldr_bins.value #bins required
    
        jump = round(intervals/bins)
        
        if all_comps == True:
                      
            
            output10.clear_output()
            
            
            x_ax_arr = t_arr[start_t:-1]
            fig = plt.figure()
            fig.title = "Time vs " +  drp_param.value + " in all compartments"
            
            for i in range(len(comp_arr)):
                
                y_ax_arr =[]
                
                if param == 'Membrane Potential (Vm)':
                    y_ax_arr.append(comp_arr[i].v_arr[start_t:-1:jump])
                elif param == 'Cl- Reversal Potential (E-Cl)':
                    y_ax_arr.append(comp_arr[i].E_cl_arr[start_t:-1:jump])
                elif param == 'K+ Reversal Potential (E-K)':
                    y_ax_arr.append(comp_arr[i].E_k_arr[start_t:-1:jump])
                elif param == 'Volume (pL)':
                    y_ax_arr.append(comp_arr[i].w_arr[start_t:-1:jump])
                elif param == 'Na+ Conc.':
                    y_ax_arr.append(comp_arr[i].na_arr[start_t:-1:jump])
                elif param == 'K+ Conc.':
                    y_ax_arr.append(comp_arr[i].k_arr[start_t:-1:jump])
                elif param == 'Cl- Conc.':
                    y_ax_arr.append(comp_arr[i].cl_arr[start_t:-1:jump])
                elif param == 'Impermeant anion Conc':
                    y_ax_arr.append(comp_arr[i].x_arr[start_t:-1:jump])         
            
                
                plt.plot(x_ax_arr, y_ax_arr,'r')
                plt.xlabel(xlabel)
                plt.ylabel(ylabel)
                
                       
                
            with output10:            
                plt.show()
        
        
        if all_comps == False:
            
            

            if param == 'Membrane Potential (Vm)':
                y_arr = comp_arr[comp_num].v_arr
            elif param == 'Cl- Reversal Potential (E-Cl)':
                y_arr = comp_arr[comp_num].E_cl_arr 
            elif param == 'K+ Reversal Potential (E-K)':
                y_arr = comp_arr[comp_num].E_k_arr 
            elif param == 'Volume (pL)':
                y_arr = comp_arr[comp_num].w_arr
            elif param == 'Na+ Conc.':
                y_arr = comp_arr[comp_num].na_arr
                col = 'm'
            elif param == 'K+ Conc.':
                y_arr = comp_arr[comp_num].k_arr
                col = 'b'
            elif param == 'Cl- Conc.':
                y_arr = comp_arr[comp_num].cl_arr
                col = 'g'
            elif param == 'Impermeant anion Conc':
                y_arr = comp_arr[comp_num].x_arr 
                col = 'c'

            elif param == 'All ion concentrations':
                y_arr1 = comp_arr[comp_num].na_arr
                y_arr2 = comp_arr[comp_num].k_arr
                y_arr3 = comp_arr[comp_num].cl_arr
                y_arr4 = comp_arr[comp_num].x_arr
                fig = plt.figure()
                fig.title = comp_arr[comp_num].name + ": all ion concentrations vs time"
                plot1 = plt.plot(x_arr[start_t:-1],y_arr1[start_t:-1])
                plot2 = plt.plot(x_arr[start_t:-1],y_arr2[start_t:-1])
                plot3 = plt.plot(x_arr[start_t:-1],y_arr3[start_t:-1])
                plot4 = plt.plot(x_arr[start_t:-1],y_arr4[start_t:-1])
                output10.clear_output()
                with output10:
                    plt.show()
                return


            fig = plt.figure()
            fig.title = comp_arr[comp_num].name+" : " + "Time (s) vs " + drp_param.value 
           

            plot = plt.plot(x_arr[start_t:-1:jump],y_arr[start_t:-1:jump],col)
            plt.xlabel(xlabel)
            plt.ylabel(ylabel)
            
            
            output10.clear_output()
            with output10:
                plt.show()

        #####


def btn_main_graph_clicked(b):
    
    
    start_t = int(sldr_start.value * 1000)
    x_arr = t_arr 
    
    bins = sldr_bins.value #bins required

    jump = round(intervals/bins)
    print(intervals,bins,jump,start_t)
    
    
    if drp_comp.value == 'ALL COMPARTMENTS':
    
            param = drp_param.value
            plot_main_graph(param,all_comps=True,jump=jump)

    
    elif drp_comp.value != 'ALL COMPARTMENTS':
        
        
        for i in range(len(comp_arr)):
            if (drp_comp.value == comp_arr[i].name):
                comp_num = i

        param = drp_param.value
        plot_main_graph(param,comp_num,all_comps=False,jump=jump)

    
    
    #if drp_comp != 'ALL COMPARTMENTS':
           
        
btn_main_graph.on_click(btn_main_graph_clicked)

#print(y_arr)

        
        


Button(button_style='success', description='Graph it!', style=ButtonStyle())

Output()

### 4.2 Boundary dynamics

Boundaries refer to the area between adjacent compartments. //
In non-isopotential neurons there must be a steady state (non-zero) flux of ions between compartments.
Negative values = ions entering the compartment.
Positive values = ions entering compartment

In [124]:
drp_bound = widgets.Dropdown(description="Boundary:")
btn_bound = widgets.Button(description = "Graph Boundary")
sldr_start = widgets.FloatSlider(description='Start time (s)',min=0,step=dt,max=total_t, value=20)
sldr_bins = widgets.FloatSlider(description='Bins',min=50,step=50,max=intervals, value=1000)

hbox_bound = widgets.HBox()
vbox_bound1 = widgets.VBox()
vbox_bound2 = widgets.VBox()
vbox_bound1.children =[drp_bound,btn_bound]
vbox_bound2.children =[sldr_start,sldr_bins]
hbox_bound.children = [vbox_bound1,vbox_bound2]


output_bound_setting = widgets.Output()


hbox_graph1 = widgets.HBox()
hbox_graph2 = widgets.HBox()
output_graph1 = widgets.Output()
output_graph2 = widgets.Output()
output_graph3 = widgets.Output()
output_graph4 = widgets.Output()
vbox_graph = widgets.VBox()
hbox_graph1.children = [output_graph1,output_graph2]
hbox_graph2.children = [output_graph3,output_graph4]
vbox_graph.children = [hbox_graph1, hbox_graph2]

with output_bound_setting:
    display(hbox_bound)

    
bound_list = []

for i in range(len(ed_arr)):
    bound_list.append(ed_arr[i].name)
    
drp_bound.options = bound_list



def btn_bound_clicked(b):
    
    
    for i in range(len(ed_arr)):
        if drp_bound.value == ed_arr[i].name:
            bound_num = i
    
    
    start_t = int(sldr_start.value * 1000)
    x_arr = t_arr 
    bins = sldr_bins.value #bins required
    jump = round(intervals/bins)
    
    
    bound = ed_arr[bound_num].name
    compA = ed_arr[bound_num].comp_a
    compB = ed_arr[bound_num].comp_b
    
    
    
    
    output_graph1.clear_output()
    output_graph2.clear_output()
    output_graph3.clear_output()
    output_graph4.clear_output()
    
    
    
    ##### Graph 1: compA - ion changes via channels
    
    
    #fig1_A.title = compA.name + " [Ion] changes across membrane"
    #tlbr1A = bqp.Toolbar(fig1_A)
    x_sc = bqp.LinearScale()
    y_sc = bqp.LinearScale()
    plt1A1 = bqp.Lines(x= t_arr[start_t:-1:jump],y=comp_arr[0].d_na_arr[start_t:-1:jump],display_legend=True,labels= [' Na'],scales = {'x': x_sc, "y": y_sc})#,color ='m', label = "Na+")
    plt1A2 = bqp.Lines(x= t_arr[start_t:-1:jump],y=comp_arr[0].d_k_arr[start_t:-1:jump],scales = {'x': x_sc, "y": y_sc})#,color ='b', label = "K+")
    plt1A3 = bqp.Lines(x= t_arr[start_t:-1:jump],y=comp_arr[0].d_cl_arr[start_t:-1:jump],scales = {'x': x_sc, "y": y_sc})#,color ='g', label = "Cl-")
    ax_x = bqp.Axis(scale=bqp.LinearScale(), label="Time(s)", tick_values = t_arr[start_t:-1:jump])
    ax_y = bqp.Axis(scale=bqp.LinearScale(), label="Concentration (mM)", orientation='vertical', side='left')
    
    fig1_A = bqp.Figure(marks=[plt1A1,plt1A2,plt1A3], axes=[ax_x, ax_y],legend_location='bottom-right')
    
  
    with output_graph1:    
        display(fig1_A)
    
    fig1_B = plt.figure()
    fig1_B.title = compB.name + " [Ion] changes across membrane"
    plt.plot(t_arr[start_t:-1:jump],compB.d_na_arr[start_t:-1:jump],'m')
    plt.plot(t_arr[start_t:-1:jump],compB.d_k_arr[start_t:-1:jump],'b')
    plt.plot(t_arr[start_t:-1:jump],compB.d_cl_arr[start_t:-1:jump],'g')
    plt.ylabel("Concentration (mM)")
    plt.xlabel("Time(s)")
    with output_graph2:    
        plt.show()
    
    fig2_A = plt.figure()
    fig2_A.title = compA.name + " [Ion] changes across compartment"
    plt.plot(t_arr[start_t:-1:jump],ed_arr[bound_num].bound_na_arr[start_t:-1:jump],'m--', label = "Na+")
    plt.plot(t_arr[start_t:-1:jump],ed_arr[bound_num].bound_k_arr[start_t:-1:jump],'b--', label ="K+")
    plt.plot(t_arr[start_t:-1:jump],ed_arr[bound_num].bound_cl_arr[start_t:-1:jump],'g--', label = "Cl-") 
    plt.ylabel("Concentration (mM)")
    plt.xlabel("Time(s)")
    with output_graph3:    
        plt.show()
       
    
    fig2_B = plt.figure()
    fig2_B.title = compB.name + " [Ion] changes across compartment"
    temp_na_arr =[]
    temp_k_arr=[]
    temp_cl_arr=[]
    for i in range(len(ed_arr[bound_num].bound_na_arr)):
        temp_na_arr.append(-1*ed_arr[bound_num].bound_na_arr[i])
        temp_k_arr.append(-1*ed_arr[bound_num].bound_k_arr[i])
        temp_cl_arr.append(-1*ed_arr[bound_num].bound_cl_arr[i])
    
    plt.plot(t_arr[start_t:-1:jump],temp_na_arr[start_t:-1:jump],'m--')
    plt.plot(t_arr[start_t:-1:jump],temp_k_arr[start_t:-1:jump],'b--')
    plt.plot(t_arr[start_t:-1:jump],temp_cl_arr[start_t:-1:jump],'g--')
    plt.ylabel("Concentration (mM)")
    plt.xlabel("Time(s)")
    with output_graph4:
        plt.show()
        

btn_bound.on_click(btn_bound_clicked)  
    
    
display(output_bound_setting,vbox_graph)
print(total_t)
#print(ed_arr[0].bound_na_arr)

Output()

60.0


Figure(axes=[Axis(label='Time(s)', scale=LinearScale()), Axis(label='Concentration (mM)', scale=LinearScale())…